1. Importing necessary libraries and unzipping the dataset

In [1]:
# Importing all the necessary libraries
import cv2
import glob
import numpy as np
from scipy.stats import multivariate_normal as gaussian
import os

In [2]:
# Unzipping the "WavingTrees" dataset. This dataset was extracted from "https://github.com/ZiyangS/BackgroundSubtraction/tree/master/WavingTrees"
# Before running this code, download the zip file of the above dataset and upload in the google colab session storage.
from zipfile import ZipFile
file_name = '/content/WavingTrees.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall('/content/WavingTrees')

2. Gaussian Function

In [3]:
def gaussian_func(X, mean, S):
  mean = np.atleast_2d(mean)
  S = np.atleast_2d(S)
  N = X.size

  temp_1 = np.linalg.det(S) ** (-0.5)
  temp_2 = np.exp(-0.5 * ((X - mean).T) @ np.linalg.inv(S) @ (X - mean))
  return (2 * np.pi) ** (-N/2) * temp_1 * temp_2

In [4]:
# Making a list of all the image filepaths so as conveniently split the training and testing dataset
images = glob.glob("/content/WavingTrees/b*.bmp")
images = sorted(images)

# Prints the total no. of images
print(len(images))

287


In [5]:
# Splitting the dataset into training and testing set
training_set_len = 220

training_set = images[:training_set_len]
testing_set = images[training_set_len:]

print(len(training_set))
print(len(testing_set))


220
67


In [6]:
# Determining the dimensions of each image 
exm_img = cv2.imread(training_set[0])
height, width, channels = exm_img.shape

print(str(height) + " x " + str(width) + " x " + str(channels))

120 x 160 x 3


In [7]:
# Definig alpha
alpha = 1.0/training_set_len

# Defining K which is the number of Gaussians as mentioned in the paper
K = 5 

# Defining the threshold to classify a pixel as a background or foreground
thres_bf = 2.5 

# Defining the threshold to check if it has a significant weight in representation of the pixel
thres_ed = 0.5

In [8]:
# Matrix for background pixels
B = np.ones((height, width),dtype=np.int64) 

# Matrix for foreground pixels
F = np.ones((height, width),dtype=np.int64)

# Weight Matrix which represents the weights of each Gaussian
weights = np.ndarray(shape=(height,width, K), dtype=np.int64) 

for h in range(height):
  for w in range(width):
    weights[h,w,:] = [1,0,0,0,0]  

# Prints the dimensions of the weight matrix and the matrix itself
print(weights.shape)
print(weights)

(120, 160, 5)
[[[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]

 [[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]

 [[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]

 ...

 [[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]

 [[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]

 [[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]]


In [9]:
# Defining the Mean matrix
mean_mat = np.ndarray(shape=(height,width,K,channels), dtype = np.float64)
for h in range(height):
  for w in range(width):
    for k in range(K):
      mean_mat[h,w,k,:] = np.zeros(3)
      mean_mat[h][w][k] = np.array(exm_img[h][w]).reshape(1,3)

# Prints the dimensions of the mean matrix and the matrix itself
print(mean_mat.shape)
print(mean_mat)

(120, 160, 5, 3)
[[[[ 57.  54.  57.]
   [ 57.  54.  57.]
   [ 57.  54.  57.]
   [ 57.  54.  57.]
   [ 57.  54.  57.]]

  [[ 58.  55.  58.]
   [ 58.  55.  58.]
   [ 58.  55.  58.]
   [ 58.  55.  58.]
   [ 58.  55.  58.]]

  [[ 59.  53.  57.]
   [ 59.  53.  57.]
   [ 59.  53.  57.]
   [ 59.  53.  57.]
   [ 59.  53.  57.]]

  ...

  [[ 20.  15.  16.]
   [ 20.  15.  16.]
   [ 20.  15.  16.]
   [ 20.  15.  16.]
   [ 20.  15.  16.]]

  [[ 11.   5.   9.]
   [ 11.   5.   9.]
   [ 11.   5.   9.]
   [ 11.   5.   9.]
   [ 11.   5.   9.]]

  [[ 11.   8.  11.]
   [ 11.   8.  11.]
   [ 11.   8.  11.]
   [ 11.   8.  11.]
   [ 11.   8.  11.]]]


 [[[243. 223. 215.]
   [243. 223. 215.]
   [243. 223. 215.]
   [243. 223. 215.]
   [243. 223. 215.]]

  [[249. 229. 216.]
   [249. 229. 216.]
   [249. 229. 216.]
   [249. 229. 216.]
   [249. 229. 216.]]

  [[252. 228. 212.]
   [252. 228. 212.]
   [252. 228. 212.]
   [252. 228. 212.]
   [252. 228. 212.]]

  ...

  [[ 61.  58.  61.]
   [ 61.  58.  61.]
   [ 61. 

In [10]:
# Defining the variance matrix
var_mat = np.ndarray(shape=(height,width,K,channels, channels), dtype = np.float64)
for h in range(height):
  for w in range(width):
    for k in range(K):
      var_mat[h,w,k,:,:] = 255*np.eye(channels)

# Prints the dimensions of the variance matrix and the matrix itself
print(var_mat.shape)
print(var_mat)

(120, 160, 5, 3, 3)
[[[[[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]]


  [[[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]]


  [[[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]

   [[255.   0.   0.]
    [  0. 255.   0.]
    [  0.   0. 255.]]]



In [11]:
# Using the thres_bf we check if the pixel could be classified as background or foreground.
def back_foreground_classifier(X, mean, var):
    m = np.mat(np.reshape(X, (3, 1)))
    n = np.mat(mean).T
    var = np.mat(var)
    d = np.sqrt((m-n).T*var.I*(m-n))
    if d < thres_bf:
        return True
    else:
        return False

In [12]:
for training_img in training_set:
  img = cv2.imread(training_img)
  for h in range(height):
    for w in range(w):
      flag = -1
      for k in range(K):
        if (back_foreground_classifier(img[h][w], mean_mat[h][w][k], var_mat[h][w][k])) is True:
          flag = k
          break
      m = np.array(img[h][w]).reshape(1,3)

      if flag != 1:
        updated_mean_mat = mean_mat[h][w][flag]
        updated_var_mat = var_mat[h][w][flag]
        m = img[h][w].astype(np.float64)
        p = gaussian.pdf(img[h][w], updated_mean_mat, updated_var_mat)

        weights[h][w] = (1-alpha)*weights[h][w]
        weights[h][w][flag] += alpha
        mean_mat[h][w][flag] += (m - updated_mean_mat)*p
        var_mat[h][w][flag] += p*(np.matmul(m-updated_mean_mat, (m-updated_mean_mat).T) - updated_var_mat)

      else:
        w = [weights[h][w][k] for k in range(K)]
        min_index = w.index(min(w))
        mean_mat[h][w][min_index] = m
        var_mat[h][w][min_index] = 255*(np.eye(3))

  for h in range(height):
        for w in range(width):
            rank = weights[h][w]*1.0/[np.linalg.norm(np.sqrt(var_mat[h][w][k])) for k in range(K)]
            rank_index = [k for k in range(K)]
            rank_index.sort(key=lambda x: -rank[x])
            weights[h][w] = weights[h][w][rank_index]
            mean_mat[h][w] =  mean_mat[h][w][rank_index]
            var_mat[h][w] = var_mat[h][w][rank_index]
            cum_weight = 0
            for index, order in enumerate(rank_index):
                cum_weight += weights[h][w][index]
                if cum_weight > thres_ed:
                    B[h][w] = index + 1
                    break
                else:
                    F[h][w] = index + 1
                    break
  print(training_img)

/content/WavingTrees/b00000.bmp
/content/WavingTrees/b00001.bmp
/content/WavingTrees/b00002.bmp
/content/WavingTrees/b00003.bmp
/content/WavingTrees/b00004.bmp
/content/WavingTrees/b00005.bmp
/content/WavingTrees/b00006.bmp
/content/WavingTrees/b00007.bmp
/content/WavingTrees/b00008.bmp
/content/WavingTrees/b00009.bmp
/content/WavingTrees/b00010.bmp
/content/WavingTrees/b00011.bmp
/content/WavingTrees/b00012.bmp
/content/WavingTrees/b00013.bmp
/content/WavingTrees/b00014.bmp
/content/WavingTrees/b00015.bmp
/content/WavingTrees/b00016.bmp
/content/WavingTrees/b00017.bmp
/content/WavingTrees/b00018.bmp
/content/WavingTrees/b00019.bmp
/content/WavingTrees/b00020.bmp
/content/WavingTrees/b00021.bmp
/content/WavingTrees/b00022.bmp
/content/WavingTrees/b00023.bmp
/content/WavingTrees/b00024.bmp
/content/WavingTrees/b00025.bmp
/content/WavingTrees/b00026.bmp
/content/WavingTrees/b00027.bmp
/content/WavingTrees/b00028.bmp
/content/WavingTrees/b00029.bmp
/content/WavingTrees/b00030.bmp
/content

In [13]:
def infer(img):
      result = np.array(img)
      for i in range(height):
          for j in range(width):
              for k in range(B[i][j]):
                  if back_foreground_classifier(img[i][j], mean_mat[i][j][k], var_mat[i][j][k]):
                      result[i][j] = [0, 0, 0]    
                      break
                  else:
                      result[i][j] = [255, 255, 255]    
                      break
      return result

In [21]:
file_list = glob.glob(r'WavingTrees/b*.bmp')
for ind, file in enumerate(file_list):
  img_1 = cv2.imread(file)
  img_2 = infer(img_1)
  # Create a folder "WavingTrees_output" in the session storage to get the output images saved in it
  cv2.imwrite('WavingTrees_output/'+'%05d'%ind+'.bmp', img_2)
  ind += 1

In [17]:
image_folder = 'WavingTrees/'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".bmp")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
# height, width, layers = frame.shape
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [22]:
image_folder_output = 'WavingTrees_output/'
video_name = 'video_output.avi'

images_output = [img for img in os.listdir(image_folder_output) if img.endswith(".bmp")]
frame = cv2.imread(os.path.join(image_folder_output, images_output[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images_output:
    video.write(cv2.imread(os.path.join(image_folder_output, image)))

cv2.destroyAllWindows()
video.release()

Comparison with the Inbuilt Function

In [24]:
inbuilt = cv2.createBackgroundSubtractorMOG2()

In [26]:
video_name = 'video_inbuilt.avi' 
image_folder = 'WavingTrees/'
images = [img for img in os.listdir(image_folder) if img.endswith(".bmp")]
for image in range(len(images)):
  frame = cv2.imread(os.path.join(image_folder, images[image]))
  height, width, channels = frame.shape
  fgmask = inbuilt.apply(frame)
  # Create a folder named "WavingTrees_output_inbuilt" in the session storage to get the output images processed by the inbuilt function saved in it.
  cv2.imwrite('WavingTrees_output_inbuilt/'+'%05d'%image+'.bmp', fgmask) 

image_folder_output_inbuilt = 'WavingTrees_output_inbuilt/'
images_output = [img for img in os.listdir(image_folder_output_inbuilt) if img.endswith(".bmp")]
frame = cv2.imread(os.path.join(image_folder_output_inbuilt, images_output[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images_output:
    video.write(cv2.imread(os.path.join(image_folder_output_inbuilt, image)))

cv2.destroyAllWindows()
video.release() 